In [1]:
import sqlite3
import pandas as pd

This is used for turning a VGS list () into a csv for other use. This section is specifically targeting species input lists...

In [2]:

# Connect to the local .db file
db_path = 'C:/ProgramData/VGSData/VGS50.db'
conn = sqlite3.connect(db_path)

# create cursor to work with database
#cursor = conn.cursor() not using this...

# species input guid
hex_value = "3B690E6214AE7443BA9FDA273CFF1752"

query = f"""Select * from SpList
left join SubTypeList on SubTypeList.FK_Type = SpList.FK_SubType
inner join SpListLink on SpListLink.FK_SpList = SpList.PK_SpList
inner join Species on Species.PK_Species = SpListLink.FK_Species
where FK_SubType = X'{hex_value}'"""

df = pd.read_sql_query(query, conn)
conn.close()
#cursor.execute(query) # not using this...
#df = pd.DataFrame(cursor.fetchall()) # or this.... 

df.head()


,PK_SpList,FK_SubType,ListName,Description,SpFilter,Definition,Locked,Active,IsHierarchical,SyncKey,...,SpeciesName,List,NewSynonym,Family,Habit,Duration,Nativity,Description,SyncKey,SyncState
0,b'L\xc6\xac\x7f\xc1\xd0]G\x9d\xc97*Q\xfc\xa5\x0f',"b"";i\x0eb\x14\xaetC\xba\x9f\xda'<\xff\x17R""",WMNF Common NNIS,None,None,None,0,1,0,547,...,Lupinus polyphyllus,NRCS,,Fabaceae,Forb/herb,Perennial,L48(N)AK(I?)CAN(N),None,23,2
1,b'L\xc6\xac\x7f\xc1\xd0]G\x9d\xc97*Q\xfc\xa5\x0f',"b"";i\x0eb\x14\xaetC\xba\x9f\xda'<\xff\x17R""",WMNF Common NNIS,None,None,None,0,1,0,547,...,Phragmites australis,NRCS,,Poaceae,"Subshrub, Shrub, Graminoid",Perennial,"L48(I,N)HI(I)CAN(N)",None,23,2
2,b'L\xc6\xac\x7f\xc1\xd0]G\x9d\xc97*Q\xfc\xa5\x0f',"b"";i\x0eb\x14\xaetC\xba\x9f\xda'<\xff\x17R""",WMNF Common NNIS,None,None,None,0,1,0,547,...,Tussilago farfara,NRCS,,Asteraceae,Forb/herb,Perennial,L48(I)CAN(I)SPM(I),None,23,2
3,b'L\xc6\xac\x7f\xc1\xd0]G\x9d\xc97*Q\xfc\xa5\x0f',"b"";i\x0eb\x14\xaetC\xba\x9f\xda'<\xff\x17R""",WMNF Common NNIS,None,None,None,0,1,0,547,...,Rosa multiflora,NRCS,,Rosaceae,"Vine, Subshrub",Perennial,L48(I)CAN(I),None,23,2
4,b'L\xc6\xac\x7f\xc1\xd0]G\x9d\xc97*Q\xfc\xa5\x0f',"b"";i\x0eb\x14\xaetC\xba\x9f\xda'<\xff\x17R""",WMNF Common NNIS,None,None,None,0,1,0,547,...,Robinia pseudoacacia,NRCS,,Fabaceae,Tree,Perennial,L48(N)CAN(I),None,23,2


In [ ]:
# Close the connection - not using this...
#cursor.close()

In [3]:
# List of unique lists found
lists = df['ListName'].unique().tolist()

for i in range(len(lists)):
    print(f"Lists found: {lists[i]}")


Lists found: WMNF Common NNIS
Lists found: WMNF Non-Alpine RFSS
Lists found: WMNF RFSS
Lists found: WMNF Shrubs
Lists found: WMNF Trees
Lists found: WMNF Understory
Lists found: WMNF VGS Plants


Export the data into csvs...

In [4]:
df.columns

Index(['PK_SpList', 'FK_SubType', 'ListName', 'Description', 'SpFilter',
       'Definition', 'Locked', 'Active', 'IsHierarchical', 'SyncKey',
       'SyncState', 'PK_SubType', 'FK_Type', 'SubListName', 'Filter',
       'Description', 'Active', 'Locked', 'SyncKey', 'SyncState',
       'PK_SpListLink', 'FK_SpList', 'FK_Species', 'CK_ParentSpecies',
       'Qualifier', 'SurrogateValue', 'IsDefault', 'Weight', 'SyncKey',
       'SyncState', 'PK_Species', 'CK_BestGuess', 'CommonName', 'SpeciesName',
       'List', 'NewSynonym', 'Family', 'Habit', 'Duration', 'Nativity',
       'Description', 'SyncKey', 'SyncState'],
      dtype='object')

In [ ]:
# select desired columns as a list
trimmed_df = pd.DataFrame(df[['ListName', 'PK_Species', 'SpeciesName', 'CommonName', 'Family', 'Habit', 'Duration', 'Nativity']])
# update name
trimmed_df = trimmed_df.rename(columns={"PK_Species":"Code"})
trimmed_df = trimmed_df.sort_values("Code")

# export each list as csv
for x in range(len(lists)):
    # go through each ListName and export a csv of each
    list_data = trimmed_df[trimmed_df['ListName']==lists[x]]
    # drop ListName column
    list_data = list_data.drop(columns=["ListName"])
    list_data.to_csv(f"createdLists/vgsListExport_{lists[x]}.csv", index = False)
    print(f"Created csv list for {lists[x]}")


Created List for WMNF Common NNIS
Created List for WMNF Non-Alpine RFSS
Created List for WMNF RFSS
Created List for WMNF Shrubs
Created List for WMNF Trees
Created List for WMNF Understory
Created List for WMNF VGS Plants
